In [54]:
import os
import pwd
import matplotlib.pyplot as plt
import math
import numpy as np
import pandas as pd
import uproot
from scipy import optimize
from scipy import constants
hbar = constants.hbar

import decay_widths as dw

def get_username():
    return pwd.getpwuid(os.getuid())[0]
os.chdir("/uboone/data/users/{}/workdir/".format(get_username()))

LAMBDA = 1e6 # MeV, energy scale (1 TeV, used in https://arxiv.org/pdf/2202.03447.pdf)

In [66]:
def match_RSE_indices(pkl_data, uproot_file,columns=[0,1,2]):
    test_run = pkl_data['run_ls'].to_numpy()
    test_subrun = pkl_data['sub_ls'].to_numpy()
    test_event = pkl_data['evt_ls'].to_numpy()

    keys = uproot_file.keys()
    data = uproot_file[keys[0]]
    keys = data.keys()
    runs = data[keys[columns[0]]].array()
    subruns = data[keys[columns[1]]].array()
    event = data[keys[columns[2]]].array()

    indices = []
    for i, run in enumerate(test_run):
        run_args = np.where(runs==run)
        subrun_args = np.where(subruns==test_subrun[i])
        event_arg = np.where(event==test_event[i])
        shared_run_sub_arg = np.intersect1d(run_args, subrun_args)
        shared_arg = np.intersect1d(shared_run_sub_arg, event_arg)
        indices.append(shared_arg[0])
    return indices


def check_matching_vtx(vtx_1, vtx_2, tolerance = 10**-7):
    diff = np.abs(vtx_1-vtx_2)
    mag_diff = diff/np.abs(vtx_2)
    warning_indices = np.where(mag_diff>tolerance)
    return warning_indices

In [69]:
mass_points = np.asarray([100,125,130,135,140,145,150,200]) # MeV
theta_values = np.asarray([5.36,5.5,5.54,5.58, 5.63, 5.68, 5.73, 5.68])*10**-4
decay_pos = ['KDAR', 'KDIF']
runs = [1,3]
shrs = [1,2]

data_pkl = {}
### Importing pkl files, variations of which are defined above
for run in runs:
    for pos in decay_pos:
        for mass in mass_points:
            for shr in shrs:
                filepath =  'HPS_uboone_analysis/BDT_inputs_pkl/AllVar_Selected_Run{}_NuMI_Signal_{}_{}_{}shr_PPFX_pred_NEW.pkl'.format(run,pos,mass,shr)
                name = "Run{}_{}_{}_{}".format(run,pos,mass,shr)
                data_pkl[name] = pd.read_pickle(filepath)


In [70]:
for sig_key, sig_data in zip(data_pkl.keys(), data_pkl.values()):
    run, K_label, mass_scalar, num_shrs = sig_key.split('_')
    if run == 'Run1': horn_config = 'fhc'
    if run == 'Run3': horn_config = 'rhc'
    
    tof_file = f"tof_{horn_config}_{mass_scalar}.root"
    train_file = f"sfn_numi_{horn_config}_generic_{mass_scalar}_ppfx_CV_train.root"

    with uproot.open("HPS_uboone_analysis/Final_v51_FHC/"+train_file) as train_data:
        keys = train_data.keys()
        train_data = train_data[keys[0]]
        
        indices_2 = match_RSE_indices(sig_data, train_data, columns=[1,2,3])
        
        keys = train_data.keys()
        train_data = train_data[keys[0]]
        keys = train_data.keys()

        
        true_nu_vtx_x = train_data['true_nu_vtx_x'].array()[indices_2].to_numpy()
        true_nu_vtx_y = train_data['true_nu_vtx_y'].array()[indices_2].to_numpy()
        true_nu_vtx_z = train_data['true_nu_vtx_z'].array()[indices_2].to_numpy()
        true_nu_vtx = [true_nu_vtx_x,true_nu_vtx_y,true_nu_vtx_z]

    with uproot.open("HPS_uboone_analysis/TOF_input_root/"+tof_file) as tof_data:
        indices_1 = match_RSE_indices(sig_data, tof_data)
        keys = tof_data.keys()
        tof_data = tof_data[keys[0]]
        keys = tof_data.keys()

        vtx_x = tof_data['vtx_x'].array()[indices_1].to_numpy()
        vtx_y = tof_data['vtx_y'].array()[indices_1].to_numpy()
        vtx_z = tof_data['vtx_z'].array()[indices_1].to_numpy()

        vtx = [vtx_x, vtx_y, vtx_z]

        bad_indices = False
        for i in range(3):
            indices = check_matching_vtx(vtx[i], true_nu_vtx[i], tolerance=1e-7)
            if np.shape(indices)[1]>0:
                bad_indices=True
            if bad_indices:
                print(f"{sig_key} error:\n")
                print(f"Files disagree on vertex coodrinates:")
                for index in indices:
                    print(f"{index}:\n vtx = {vtx[i][index]}\n tru_nu_vtx = {true_nu_vtx[i][index]}\n diff = {np.abs(vtx[i][index]-true_nu_vtx[i][index])}")


        if not bad_indices:
            print(f"{sig_key} ran successfully.")
            tof = tof_data['proper_tof'].array()[indices_1].to_numpy()

Run1_KDAR_100_1 ran successfully.
Run1_KDAR_100_2 ran successfully.
Run1_KDAR_125_1 ran successfully.
Run1_KDAR_125_2 ran successfully.
Run1_KDAR_130_1 ran successfully.
Run1_KDAR_130_2 ran successfully.
Run1_KDAR_135_1 ran successfully.
Run1_KDAR_135_2 ran successfully.
Run1_KDAR_140_1 ran successfully.
Run1_KDAR_140_2 ran successfully.
Run1_KDAR_145_1 ran successfully.
Run1_KDAR_145_2 ran successfully.
Run1_KDAR_150_1 ran successfully.
Run1_KDAR_150_2 ran successfully.
Run1_KDAR_200_1 ran successfully.
Run1_KDAR_200_2 ran successfully.
Run1_KDIF_100_1 ran successfully.
Run1_KDIF_100_2 ran successfully.
Run1_KDIF_125_1 ran successfully.
Run1_KDIF_125_2 ran successfully.
Run1_KDIF_130_1 ran successfully.
Run1_KDIF_130_2 ran successfully.
Run1_KDIF_135_1 ran successfully.
Run1_KDIF_135_2 ran successfully.
Run1_KDIF_140_1 ran successfully.
Run1_KDIF_140_2 ran successfully.
Run1_KDIF_145_1 ran successfully.
Run1_KDIF_145_2 ran successfully.
Run1_KDIF_150_1 ran successfully.
Run1_KDIF_150_